In [1]:
from agent.scrape import scrape_sklearn_website

In [2]:
# sklearn_data = scrape_sklearn_website()

In [3]:
import json

with open("data/sklearn_function_openai.json","r") as jsonfile:
    sklearn_data = json.load(jsonfile)

In [4]:
from agent.utils import build_no_summary_graph,get_parents_dict

In [5]:
sklearn_graph = build_no_summary_graph(sklearn_data)

In [6]:
parents_dict = get_parents_dict(sklearn_graph)

In [7]:
parents_dict['sklearn.model_selection']['sklearn.model_selection#Splitters']

['K-fold iterator variant with non-overlapping groups. Each group will appear exactly once in the test set across all folds (the\nnumber of distinct groups has to be at least equal to the number of folds). The folds are approximately balanced in the sense that the number of\nsamples is approximately the same in each test fold. Read more in the User Guide. For visualisation of cross-validation behaviour and\ncomparison between common scikit-learn split methods\nrefer to Visualizing cross-validation behavior in scikit-learn Number of folds. Must be at least 2. Changed in version 0.22: n_splits default value changed from 3 to 5. See also For splitting the data according to explicit domain-specific stratification of the dataset. Takes class information into account to avoid building folds with imbalanced class proportions (for binary or multiclass classification tasks). Notes Groups appear in an arbitrary order throughout the folds.  Get metadata routing of this object. Please check User G

## LCEL TO SUMMARIZE THE DATA

In [8]:
# ans = summary_chain.invoke({"descriptions":"\n\n".join(parents_dict['sklearn']['sklearn#defaults'])})

In [9]:
# ans = await summary_chain.abatch([{"descriptions":"\n\n".join(parents_dict['sklearn']['sklearn#defaults'])}],config={"max_concurrency": 5})
# ans

In [21]:
from typing import List
def split_description(parent_text:List[str],MAX_WORDS:int=500):
        split_s = []
        running_num_words = 0
        curr_func_string = ""
        for txt in parent_text:
            num_words = len(txt.split(" "))
            running_num_words += num_words
            if running_num_words > MAX_WORDS:
                running_num_words = num_words
                split_s.append(curr_func_string)
                curr_func_string = txt
            else:
                curr_func_string += txt + "\n"
        if split_s == []:
            split_s.append(curr_func_string)
        split_s = [s for s in split_s if s!=""]
        return split_s

In [22]:
parent_child_dict = {pn:{} for pn in parents_dict}

for parent_name,child_nodes in parents_dict.items():
    for child_name,child_texts in child_nodes.items():
        child_split_list = split_description(child_texts,500)
        parent_child_dict[parent_name].update({child_name:child_split_list})        

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv,find_dotenv
# from langchain_openai_limiter import LimitAwaitChatOpenAI, ChooseKeyChatOpenAI

load_dotenv(find_dotenv(),override=True)
# chat_model_limit_await = LimitAwaitChatOpenAI(
#     chat_openai=model,
#     limit_await_timeout=60.0,
#     limit_await_sleep=0.1,
# )
# chat_model_key_choose = ChooseKeyChatOpenAI(
#     chat_openai=chat_model_limit_await,
#     openai_api_keys=[
#         os.environ["OPENAI_API_KEY0"],
#         os.environ["OPENAI_API_KEY1"],
#     ]
# )

summary_prompt = ChatPromptTemplate.from_template(
"""
You are given a list of descriptions of different functions separated by new paragraph.
Your task is to summarize all the text into coherent and detailed summary that covers all the functions descriptions.
Be very diligent and make sure that no function description is left out of the final summary. 

Follow the following format

List of function descriptions: list of function descriptions to be summarized
Summary: summary of all the function descriptions

-----

List of functions descriptions: {descriptions}

Summary: 
"""
)

parent_summary_dict = {pn:{} for pn in parents_dict}

batch_list = []
for parent_name,child_node in parent_child_dict.items():
    for child_name,child_texts in child_node.items():
        for ctext in child_texts:
            batch_list.append({f"{parent_name}-->{child_name}":ctext})


description_list = [{"descriptions":list(desc.values())[0]} for desc in batch_list]

BATCH_SIZE = 5
start_key = '0'
summaries_list = []
for start in tqdm(range(0,len(description_list),BATCH_SIZE),desc="Embedding rows"):
    end = min(start+BATCH_SIZE,len(description_list))
    model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.environ[f'OPENAI_API_KEY{start_key}'],max_retries=5)
    summary_chain = summary_prompt | model | StrOutputParser()
    curr_description_list = description_list[start:end]
    curr_summary_list = summary_chain.batch(curr_description_list,config={"max_concurrency":5})
    summaries_list.extend(curr_summary_list)



summary_list = summary_chain.batch(description_list,config={"max_concurrency":5})
from tqdm import tqdm

pbar = tqdm(total=len(batch_list))
for sl in summary_list:
    bl = batch_list[sl[0]]
    parent_name = list(bl.keys())[0].split("-->")[0]
    child_name = list(bl.keys())[0].split("-->")[1]
    if child_name in parent_summary_dict[parent_name]:
        parent_summary_dict[parent_name][child_name].append(sl[1])
    else:
        parent_summary_dict[parent_name].update({child_name:[sl[1]]})
    pbar.update(1)

In [20]:
len(description_list),len(batch_list)

(353, 353)

In [18]:
ans = summary_chain.batch_as_completed(description_list,config={"max_concurrency":10})

In [14]:
for i in ans:
    print(i)

(0, "The global scikit-learn configuration offers various options for customization. \n- The configuration includes settings for validation of finiteness, size of temporary arrays, printing of non-default parameters, display of estimators, row vectors per chunk, accelerated pairwise-distances reduction backend, Array API dispatching, output format of transform and fit_transform, metadata routing, validation of hyper-parameters' types and values, and retrieval of current configuration values.\n- Users can choose to skip validation for finiteness for faster processing, set a limit for temporary array size, control the printing of non-default parameters, display estimators as diagrams or text, adjust the number of row vectors per chunk, use the accelerated pairwise-distances reduction backend, enable Array API dispatching, configure output formats, enable metadata routing, and disable validation of hyper-parameters' types and values.\n- These configurations can be customized globally and 

In [ ]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125",api_key=os.environ['OPENAI_API_KEY'],max_retries=5,model_kwargs={"batch_size":20})

In [24]:
# def build_graph(sklearn_graph):
from copy import deepcopy
import re
import ast
embed_docs = []
embed_metadata = []

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
    length_function=len,
    is_separator_regex=False,
    chunk_size=2500,
    chunk_overlap=400)
with open("parent_summary.json","r") as jsonfile:
    parent_summary_dict = json.load(jsonfile)

splits = text_splitter.split_text(parent_summary_dict['sklearn.linear_model'])
with open("parent_summary.json","r") as jsonfile:
    parent_summary_dict = json.load(jsonfile)

def clean_text(s:str):
    s = re.sub("\n"," ",s)
    s = json.dumps(s)
    return ast.literal_eval(s)
for node,attr in sklearn_graph.nodes(data=True):
    type = attr.get('type')
    
    if type == 'function_node':
        ftext = attr['function_text']
        if ftext == "": continue
        embed_docs.append(clean_text(attr['function_text']))
        for k,v in attr.items():
            if not isinstance(v,str):
                attr[k] = str(v)
        embed_metadata.append(attr)
    elif type == 'parent_node':
        parent_summary = parent_summary_dict[node]
        parent_summary_text_split = text_splitter.split_text(parent_summary)
        for k,v in attr.items():
            if not isinstance(v,str):
                attr[k] = str(v)
        attr.update({"name":node})
        for parent_splits in parent_summary_text_split:
            embed_docs.append(clean_text(parent_splits))
            embed_metadata.append(attr)
    elif type == 'sub_level_node':
        for ctext in attr['child_texts']:
            embed_docs.append(clean_text(ctext))
            sub_level_attr = deepcopy(attr)
            del sub_level_attr['child_texts']
            for k,v in sub_level_attr.items():
                if not isinstance(v,str):
                    sub_level_attr[k] = str(v)
            sub_level_attr.update({"name":node})
            embed_metadata.append(sub_level_attr)

In [25]:
len(embed_docs),len(embed_metadata)

(1303, 1303)

In [26]:
embed_metadata[500]

{'type': 'sub_level_node',
 'trail': 'sklearn.exceptions',
 'name': 'sklearn.exceptions#defaults'}

In [27]:
import chromadb.utils.embedding_functions as embedding_functions
import chromadb
from chromadb.utils.batch_utils import create_batches
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    wait_fixed
)
@retry(wait=wait_random_exponential(min=1,max=60),stop=stop_after_attempt(6))
def build_database(offset:int,docs, metadata, api_key):
    database_path = "SKLEARN_DB"
    collection_name = "sklearn"
    load_dotenv(find_dotenv(), override=True)
    emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key, model_name="text-embedding-3-small"
    )

    client = chromadb.PersistentClient(path=database_path)
    sklearn_collection = client.get_or_create_collection(
        name=collection_name, embedding_function=emb_fn
    )

    sklearn_ids = [f"id{offset+i}" for i in range(len(docs))]
    all_ids = sklearn_collection.get()['ids']
    if all(skids in all_ids for skids in sklearn_ids):
        print(f"All ids already present in database {offset}")
        return 
    batches = create_batches(
        api=client, ids=sklearn_ids, documents=docs, metadatas=metadata
    )
    for batch in batches:
        sklearn_collection.add(ids=batch[0], documents=batch[3], metadatas=batch[2])
    return sklearn_collection

In [28]:
from dotenv import load_dotenv,find_dotenv
import os

load_dotenv(find_dotenv(),override=True)

def create_embeddings(embed_docs,embed_metadata):
    BATCH_SIZE = 200
    # pbar = tqdm(total=len(embed_docs)//BATCH_SIZE,)
    for start in range(0,len(embed_docs),BATCH_SIZE):
        end = min(start+BATCH_SIZE,len(embed_docs))
        # sklearn_ids = [f"id{start+i}" for i in range()]
        curr_docs = embed_docs[start:end]
        curr_metadata = embed_metadata[start:end]
        build_database(start,curr_docs[:len(curr_docs)//2],curr_metadata[:len(curr_metadata)//2],os.environ['OPENAI_API_KEY1'])
        build_database(start+BATCH_SIZE//2,curr_docs[len(curr_docs)//2:],curr_metadata[len(curr_metadata)//2:],os.environ['OPENAI_API_KEY1'])
        # print("Sleeping...")
        # time.sleep(60)
        print(f"Done for {start}-{end}/{len(embed_docs)}")

create_embeddings(embed_docs,embed_metadata)

Done for 0-200/1303
Done for 200-400/1303
Done for 400-600/1303
Done for 600-800/1303
Done for 800-1000/1303
Done for 1000-1200/1303
Done for 1200-1303/1303


In [29]:
database_path = "SKLEARN_DB"
emb_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=os.environ["OPENAI_API_KEY0"], model_name="text-embedding-3-small"
    )

client = chromadb.PersistentClient(path=database_path)
sklearn_collection = client.get_collection(
        name="sklearn", embedding_function=emb_fn
    )

In [30]:
all_ids = sklearn_collection.get()['ids']

In [31]:
import ast
d = sklearn_collection.get(
    where = {"type":"sub_level_node"},
)
d['metadatas'][0]

{'name': 'sklearn.model_selection#Model validation',
 'trail': 'sklearn.model_selection',
 'type': 'sub_level_node'}

In [102]:
import dspy
def generate_pairs(list1, list2):
    pairs = []
    for l1 in list1:
        for l2 in list2:
            curr_trail = l1
            curr_trail += f"-->{l2}"
            pairs.append(curr_trail)
    return [pairs]


def generate_pairs_recursive(trail_list):
    if len(trail_list) == 1:
        return trail_list[0]
    curr_pairs = generate_pairs(trail_list[-2], trail_list[-1])
    modified_trail_list = trail_list[:-2] + curr_pairs
    return generate_pairs_recursive(modified_trail_list)


def get_trail_list_pairs(trail_list_pairs, metadata_name="trail"):
    if len(trail_list_pairs) == 1:
        trail_where_clause = {metadata_name: {"$eq": trail_list_pairs[0]}}
    elif len(trail_list_pairs) > 1:
        trail_where_clause = {
            "$or": [{metadata_name: {"$eq": t}} for t in trail_list_pairs]
        }
    return trail_where_clause

class FirstSecondLevel(dspy.Signature):
    "You are given a list of keys and values separated by semicolon."
    "Based on the query, you have to output the key that is most relevant to the question separated by semicolon."
    "Be precise and output only the relevant key or keys from the provided keys only."
    "Don't include any other information"

    query = dspy.InputField(prefix="Query which you need to classify: ", format=str)
    keys_values = dspy.InputField(prefix="Keys and Values: ", format=str)
    output = dspy.OutputField(
        prefix="Relevant Key(s): ",
        format=str,
        desc="relevant keys separated by semicolon",
    )
llm = dspy.OpenAI(model="gpt-3.5-turbo-0125",max_tokens=512)
dspy.settings.configure(lm=llm)

class SklearnAgentChroma(dspy.Module):
    def __init__(self, collection):
        super().__init__()
        self.collection = collection
        self.firstSecondLevel = dspy.Predict(FirstSecondLevel)

    def __call__(self, *args, **kwargs):
        return super().__call__(*args, **kwargs)

    def forward(self, query: str):
        query_emb = emb_fn([query])[0]

        # Parent level querying
        parent_level = self.collection.query(
            query_embeddings=query_emb,
            where={
                "type": {"$eq": "parent_node"},
            },
            n_results=3,
        )
        parent_level_str = ""
        for parent_level_docs,parent_level_metadata in zip(parent_level['documents'][0],parent_level["metadatas"][0]):
            parent_level_str += f"{parent_level_metadata['name']}: {parent_level_docs}\n\n"

        parent_level_answer = self.firstSecondLevel(
            query=query, keys_values=parent_level_str
        ).output
        print(parent_level_str, parent_level_answer)
        trail_list = [parent_level_answer.split(";")]
        trail_list = list(set(trail_list[0]))
        trail_list_pairs = generate_pairs_recursive([trail_list])

        trail_where_clause = get_trail_list_pairs(trail_list_pairs)

        sub_level = self.collection.query(
            query_embeddings=query_emb,
            where={
                "$and": [
                    trail_where_clause,
                    {"type": {"$eq": "sub_level_node"}},
                ]
            },
            n_results=5,
        )

        sub_level_str = ""
        for sub_level_docs,function_level_metadata in zip(sub_level['documents'][0],sub_level["metadatas"][0]):
            sub_level_str += f"{function_level_metadata['name']}: {sub_level_docs}\n\n"
        print(sub_level_str)
        sub_level_answer = self.firstSecondLevel(
            query=query, keys_values=sub_level_str
        ).output
        print(sub_level_answer)
        sub_level_list = [sla.split("#")[-1] for sla in sub_level_answer.split(";")]
        sub_level_list = list(set(sub_level_list))
        function_list = generate_pairs_recursive([trail_list_pairs,sub_level_list])
        function_where_clause = get_trail_list_pairs(function_list)
        print(function_where_clause)
        functions = self.collection.query(
            query_embeddings=query_emb,
            where={
                "$and": [
                    function_where_clause,
                    {"type": {"$eq": "function_node"}},
                ]
            },
            n_results=1
        )
        return functions
sklearn_chroma = SklearnAgentChroma(sklearn_collection)
funcs = sklearn_chroma("How to do empirical covariance of X and y?")

sklearn.covariance: computing Mahalanobis distances, log-likelihood of test data under a Gaussian model, setting parameters of the estimator, and requesting metadata routing. These functions provide a comprehensive set of tools for covariance estimation and analysis. The GraphicalLassoCV function is used for sparse inverse covariance estimation with a cross-validated choice of the l1 penalty. It allows for the selection of the penalization parameter and provides various outputs such as the estimated mean, covariance matrix, precision matrix, and penalization parameter selected. The function uses a grid search approach to find the optimal penalization parameter and can handle ill-conditioned systems. Additionally, it provides options for setting the tolerance for convergence, the solver type, number of iterations, and parallel processing. The function also offers methods for computing the Mean Squared Error between covariance estimators, fitting the covariance model, computing Mahalanob

In [103]:
funcs['metadatas'][0][0]['function_calling']

"{'name': 'empirical_covariance', 'descriptions': 'Compute the Maximum likelihood covariance estimator. Data from which to compute the covariance estimate. If True, data will not be centered before computation.\\nUseful when working with data whose mean is almost, but not exactly\\nzero.\\nIf False, data will be centered before computation. Empirical covariance (Maximum Likelihood Estimator). Examples', 'parameters': {'type': 'object', 'properties': {'X': {'type': 'array', 'description': 'ndarray of shape (n_samples, n_features). Data from which to compute the covariance estimate.\\n'}, 'assume_centered': {'type': 'boolean', 'description': 'bool, default=False. If True, data will not be centered before computation.\\nUseful when working with data whose mean is almost, but not exactly\\nzero.\\nIf False, data will be centered before computation.\\n'}}, 'required': ['X']}}"

In [97]:
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document

class FirstSecondLevel_(dspy.Signature):
    "You are given a list of keys and values separated by semicolon."
    "Based on the query, you have to output the key that is most relevant to the question separated by semicolon."
    "Be precise and output only the relevant key or keys from the provided keys only."
    "Don't include any other information and don't answer None and N/A"

    query = dspy.InputField(prefix="Query which you need to classify: ", format=str)
    keys_values = dspy.InputField(prefix="Keys and Values: ", format=str)
    output = dspy.OutputField(
        prefix="Relevant Key(s): ",
        format=str,
        desc="relevant keys separated by semicolon",
    )

class SklearnAgentBM25(dspy.Module):
    def __init__(self, collection):
        super().__init__()
        self.collection = collection
        self.firstSecondLevel = dspy.Predict(FirstSecondLevel_)
        self.parent_langchain_docs = []
        parent_level = self.collection.get(
            where={
                "type": {"$eq": "parent_node"},
            }
        )
        for doc,metadata in zip(parent_level['documents'],parent_level['metadatas']):
            self.parent_langchain_docs.append(Document(page_content=doc,metadata=metadata))
    def __call__(self, *args, **kwargs):
        return super().__call__(*args, **kwargs)
    
    def BM25RetrieverLangchain(self,query:str,node_type:str='parent_node',trail_where_clause:dict={}):

        assert node_type in ['parent_node','function_node','sub_level_node'], "type must be 'parent_node' or 'function_node' or 'sub_level_node'"
        if node_type != 'parent_node' and trail_where_clause=={}:
            raise ValueError("trail_where_clause must be a dict for function type")
        
        if node_type == 'parent_node':
            bm25_retriever = BM25Retriever.from_documents(
                self.parent_langchain_docs, k=5, preprocess_func=(lambda x: x.lower())
                )
            parent_bm25_docs = bm25_retriever.invoke(query.lower())
            return parent_bm25_docs
        else:
            function_level = self.collection.get(
            where={
                "$and": [
                    trail_where_clause,
                    {"type": {"$eq": node_type}},
                ]
            },
             )
            function_langchain_docs = []
            for doc,metadata in zip(function_level['documents'],function_level['metadatas']):
                function_langchain_docs.append(Document(page_content=doc,metadata=metadata))
            bm25_retriever = BM25Retriever.from_documents(
                function_langchain_docs, k=3, preprocess_func=(lambda x: x.lower())
            )
            bm25_docs = bm25_retriever.invoke(query.lower())
            return bm25_docs
       
    def forward(self, query: str):
        parent_bm25_docs = self.BM25RetrieverLangchain(query,node_type="parent_node")
        parent_level_str = ""
        for parent_doc in parent_bm25_docs:
            parent_level_str+=f"{parent_doc.metadata['name']}: {parent_doc.page_content}\n"
        
        parent_level_answer = self.firstSecondLevel(
            query=query, keys_values=parent_level_str
        ).output
        print(parent_level_str,parent_level_answer)
        trail_list = [parent_level_answer.split(";")]
        trail_list = list(set(trail_list[0]))
        trail_list_pairs = generate_pairs_recursive([trail_list])

        trail_where_clause = get_trail_list_pairs(trail_list_pairs)

        sub_level_docs = self.BM25RetrieverLangchain(query,node_type='sub_level_node',trail_where_clause=trail_where_clause)
        sub_level_str = ""
        for function_doc in sub_level_docs:
            sub_level_str+=f"{function_doc.metadata['name']}: {function_doc.page_content}\n"
        print(sub_level_str)
        sub_level_answer = self.firstSecondLevel(
            query=query, keys_values=sub_level_str
        ).output
        
        sub_level_list = [sla.split("#")[-1] for sla in sub_level_answer.split(";")]
        sub_level_list = list(set(sub_level_list))
        function_list = generate_pairs_recursive([trail_list_pairs,sub_level_list])
        function_where_clause = get_trail_list_pairs(function_list)
        print(function_where_clause)
        functions = self.BM25RetrieverLangchain(query,node_type="function_node",trail_where_clause=function_where_clause)
        return functions
sklearn_bm25 = SklearnAgentBM25(sklearn_collection)

funcs = sklearn_bm25("I want to linear discriminant analysis with X and y?")

sklearn.model_selection: On the other hand, the Leave-P-Out cross-validator also provides train/test indices to split data into train/test sets, resulting in testing on all distinct samples of size p while the remaining n - p samples form the training set in each iteration. It is important to note that LeavePOut(p) is not equivalent to KFold(n_splits=n_samples // p) as it creates non-overlapping test sets. Similar to Leave-One-Out, this method can be costly for large datasets, and it is advised to use KFold, StratifiedKFold, or ShuffleSplit instead.
sklearn.neighbors: be specified, such as 'ball_tree', 'kd_tree', or 'brute'. The weight function used in prediction can be set to 'uniform', 'distance', or a user-defined function. Additional parameters like leaf size, power parameter for the Minkowski metric, and metric for distance computation can also be adjusted. The model can be fitted with training data and used to predict target values. The coefficient of determination (R^2) can be c

In [98]:
funcs

[Document(page_content='Create a callable to select columns to be used with ColumnTransformer. make_column_selector can select columns based on datatype or the columns name with a regex. When using multiple selection criteria, all criteria must match for a column to be selected. For an example of how to use make_column_selector within a ColumnTransformer to select columns based on data type (i.e. dtype), refer to Column Transformer with Mixed Types. Name of columns containing this regex pattern will be included. If None, column selection will not be selected based on pattern. A selection of dtypes to include. For more details, see pandas.DataFrame.select_dtypes. A selection of dtypes to exclude. For more details, see pandas.DataFrame.select_dtypes. Callable for column selection to be used by a ColumnTransformer. See also Class that allows combining the outputs of multiple transformer objects used on column subsets of the data into a single feature space. Examples Callable for column se